# Suspend and a virtual stack

In [ ]:
suspend fun SequenceScope<Int>.collatzGenerator(start: Int) {
    var n = start

    while (true) { // function ENTRY, loop start
        yield(n) // suspend point
        if (n % 2 == 0) { // CONTINUE here after yield *returns*
            n /= 2
        } else {
            n = 3 * n + 1
        } // goto entry
    }
}


In [ ]:
// To use our generator, we need a special *wrapper* function, but it is JVM magic so I won't decompile it
fun collatzSequence(start: Int) = sequence { collatzGenerator(start) }

In [ ]:
// Create a sequence, go UNTIL we find a 1 (aka 1->4->2->1 loop), print all
collatzSequence(12344).takeWhile { it != 1 }.forEach(::println)

# So, how does it work?
It could be a normal thread, but a thread is expensive and switching between threads is slow

Let's decompile it

In [10]:
/**
* This works pretty well as long as we don't want to invoke another suspend function from it
*/
class CollatzSequenceDecompiled(start: Int) {
    enum class State {
        ENTRY, CONTINUE
    }

    private var n: Int = start
    private var label = State.ENTRY

    fun getNextValue(): Int {
        while (true) {
            when (label) {
                State.ENTRY -> {
                    label = State.CONTINUE
                    return n
                }
                State.CONTINUE -> {
                    if (n % 2 == 0) {
                        n /= 2
                    } else {
                        n = 3 * n + 1
                    }
                    label = State.ENTRY
                }
            }
        }
    }
}

In [ ]:
// But what if we want to do that?